In [1]:
## imports 
import pandas as pd
import numpy as np
import yaml
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

# comment these out if you don't have plotnine--not essential here/only used once
import plotnine
from plotnine import *

## way to connect to mysql 
## if you need to install
## uncomment this line:
#! pip install mysql-connector-python
import mysql.connector

## print mult
## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## function to feed path name to load
## credentials
def load_creds(path: str):
    with open(path, 'r') as stream:
        try:
            creds = yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            print(exc)
    return(creds)


/Users/vaanigupta/opt/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


# 0. Preliminary: define connection and read sample of data

In [11]:
creds = load_creds("09_db_cred.yaml")

In [12]:
## connect to the database
cnx = mysql.connector.connect(user=creds['practice_database']['user'], 
                            password=creds['practice_database']['password'],
                            port=creds['practice_database']['port'],
                            database= creds['practice_database']['database'],
                            host = creds['practice_database']['host'])
cnx

In [13]:
## define a query to pull first 5 rows
## from caseinit table
sample_case_q = """ 
select 
    * 
from caseinit
limit 5
"""

In [14]:
## feed read sql query the query
## and my database connection
read_sample_d = pd.read_sql_query(sample_case_q, cnx)

read_sample_d

,CASE_ID,CASE_PARTICIPANT_ID,RECEIVED_DATE,OFFENSE_CATEGORY,PRIMARY_CHARGE_FLAG,CHARGE_ID,CHARGE_VERSION_ID,DISPOSITION_CHARGED_OFFENSE_TITLE,CHARGE_COUNT,DISPOSITION_DATE,...,INCIDENT_END_DATE,LAW_ENFORCEMENT_AGENCY,LAW_ENFORCEMENT_UNIT,ARREST_DATE,FELONY_REVIEW_DATE,FELONY_REVIEW_RESULT,ARRAIGNMENT_DATE,UPDATED_OFFENSE_CATEGORY,is_in_diversion,in_chicago
0,198055620664,85937621020,08/15/1984 12:00:00 AM,PROMIS Conversion,False,1242195814523,155656315869,FIRST DEGREE MURDER,2,12/17/2014 12:00:00 AM,...,,CHICAGO POLICE DEPT,,08/15/1984 12:00:00 AM,08/15/1984 12:00:00 AM,Charge(S) Approved,09/21/1984 12:00:00 AM,PROMIS Conversion,False,None
1,198055620664,85937621020,08/15/1984 12:00:00 AM,PROMIS Conversion,False,1242198287388,131513547452,HOME INVASION,14,12/17/2014 12:00:00 AM,...,,CHICAGO POLICE DEPT,,08/15/1984 12:00:00 AM,08/15/1984 12:00:00 AM,Charge(S) Approved,09/21/1984 12:00:00 AM,PROMIS Conversion,False,None
2,198055620664,85937621020,08/15/1984 12:00:00 AM,PROMIS Conversion,False,1242351605056,176626576281,FIRST DEGREE MURDER,4,12/17/2014 12:00:00 AM,...,,CHICAGO POLICE DEPT,,08/15/1984 12:00:00 AM,08/15/1984 12:00:00 AM,Charge(S) Approved,09/21/1984 12:00:00 AM,PROMIS Conversion,False,None
3,198055620664,85937621020,08/15/1984 12:00:00 AM,PROMIS Conversion,False,1242352841488,176617824190,FIRST DEGREE MURDER,5,12/17/2014 12:00:00 AM,...,,CHICAGO POLICE DEPT,,08/15/1984 12:00:00 AM,08/15/1984 12:00:00 AM,Charge(S) Approved,09/21/1984 12:00:00 AM,PROMIS Conversion,False,None
4,198055620664,85937621020,08/15/1984 12:00:00 AM,PROMIS Conversion,False,1242356550787,131238606761,HOME INVASION,13,12/17/2014 12:00:00 AM,...,,CHICAGO POLICE DEPT,,08/15/1984 12:00:00 AM,08/15/1984 12:00:00 AM,Charge(S) Approved,09/21/1984 12:00:00 AM,PROMIS Conversion,False,None


# 1. Row and column filtering

In [15]:
## 1.1 Pulling columns

In [16]:
## select specific columns
pull_id_q = """
select 
    CASE_ID, 
    CASE_PARTICIPANT_ID
from caseinit
"""

id_d = pd.read_sql_query(pull_id_q,
                             cnx)
id_d.head()

,CASE_ID,CASE_PARTICIPANT_ID
0,198055620664,85937621020
1,198055620664,85937621020
2,198055620664,85937621020
3,198055620664,85937621020
4,198055620664,85937621020


## 1.2 subsetting rows

In [17]:
## select specific rows using where
pull_id_age_q = """
select 
    CASE_ID, 
    CASE_PARTICIPANT_ID, 
    AGE_AT_INCIDENT
from caseinit
where AGE_AT_INCIDENT > 40
"""

id_age = pd.read_sql_query(pull_id_age_q,
                cnx)
id_age.head()

,CASE_ID,CASE_PARTICIPANT_ID,AGE_AT_INCIDENT
0,205411772533,106247326235,49.0
1,208597923533,114147409332,43.0
2,209306582865,122563667565,47.0
3,209754034523,117614073313,48.0
4,210275284632,117990102885,45.0


In [18]:
## select specific rows using IN
pull_id_r_q = """
select 
    CASE_ID, 
    CASE_PARTICIPANT_ID, 
    RACE
from caseinit
where RACE in ("Black", "HISPANIC")
"""

id_r = pd.read_sql_query(pull_id_r_q,
                cnx)
id_r.RACE.value_counts()

RACE
Black       181219
HISPANIC      6098
Name: count, dtype: int64

## 1.3 Creating new columns

In [20]:
## create new col 
## using case when logic
compare_charge_q = """
SELECT *,
       CASE
         WHEN offense_category = updated_offense_category THEN 'Same offense'
         ELSE 'Diff offense'
       END AS charge_update
FROM   caseinit 
"""

compare_charge_d = pd.read_sql_query(compare_charge_q, cnx)

compare_charge_d.charge_update.value_counts()
samp = compare_charge_d.groupby('charge_update').sample(n = 2, random_state = 919)
samp[['OFFENSE_CATEGORY', 'UPDATED_OFFENSE_CATEGORY', 'charge_update']]

charge_update
Same offense    239276
Diff offense     33018
Name: count, dtype: int64

,OFFENSE_CATEGORY,UPDATED_OFFENSE_CATEGORY,charge_update
214095,DUI,Aggravated DUI,Diff offense
14131,Aggravated Battery Police Officer,Disarming Police Officer,Diff offense
75493,Burglary,Burglary,Same offense
161184,Narcotics,Narcotics,Same offense


## 1.4 Code that deliberately throws error due to creating var in same step as row filter

In [19]:
## create new col 
## using case when logic
## and then row filter
## throws error bc doesn't
## know that charge_update exists
compare_charge_filt_q = """
SELECT *,
       CASE
         WHEN offense_category = updated_offense_category THEN 'Same offense'
         ELSE 'Diff offense'
       END AS charge_update
FROM   caseinit
WHERE  charge_update = 'Diff offense' 
"""

compare_charge_filt_d = pd.read_sql_query(compare_charge_filt_q, cnx)


DatabaseError: Execution failed on sql '
SELECT *,
       CASE
         WHEN offense_category = updated_offense_category THEN 'Same offense'
         ELSE 'Diff offense'
       END AS charge_update
FROM   caseinit
WHERE  charge_update = 'Diff offense' 
': 1054 (42S22): Unknown column 'charge_update' in 'where clause'

In [21]:
## row filtering using not equal syntax
compare_charge_q = """
select 
    *
    from caseinit
where OFFENSE_CATEGORY <> UPDATED_OFFENSE_CATEGORY
"""

test = pd.read_sql_query(compare_charge_q, cnx)
test[['OFFENSE_CATEGORY', 'UPDATED_OFFENSE_CATEGORY']].head()

,OFFENSE_CATEGORY,UPDATED_OFFENSE_CATEGORY
0,Attempt Homicide,Domestic Battery
1,Attempt Homicide,Domestic Battery
2,Armed Violence,UUW - Unlawful Use of Weapon
3,Domestic Battery,Homicide
4,Domestic Battery,Homicide


# 2. Subqueries and aggregation: one dataset

## 2.1 Row filtering using subquery

In [22]:
## row filtering using subquery
compare_charge_sub_q = """
SELECT *
FROM   caseinit
       INNER JOIN (SELECT case_id    AS cid,
                          case_participant_id AS cpid,
                          CASE
                            WHEN offense_category = updated_offense_category
                          THEN
                            'Same offense'
                            ELSE 'Diff offense'
                          END                 AS charge_update
                   FROM   caseinit) AS tmp
               ON tmp.cid = caseinit.case_id
                  AND tmp.cpid = caseinit.case_participant_id
WHERE  charge_update = "diff offense" 
"""

diff_c = pd.read_sql_query(compare_charge_sub_q, cnx)
diff_c[['OFFENSE_CATEGORY', 'UPDATED_OFFENSE_CATEGORY', 'charge_update']].head()

,OFFENSE_CATEGORY,UPDATED_OFFENSE_CATEGORY,charge_update
0,Aggravated Battery Police Officer,UUW - Unlawful Use of Weapon,Diff offense
1,DUI,Aggravated DUI,Diff offense
2,DUI,Aggravated DUI,Diff offense
3,DUI,Aggravated DUI,Diff offense
4,DUI,Aggravated DUI,Diff offense


## 2.2 Longer example: racial disparities in who's offered diversion

-  Find the five most common offenses in the `caseinit` table
- For those five most common offenses, find the percent of Black defendants whose cases are diverted and the percent of White defendants whose cases are diverted


In [23]:
## top 5 offenses
t5 = """
SELECT updated_offense_category,
       Count(*) AS count_offense
FROM   caseinit
WHERE  race IN ( "black", "white" )
GROUP  BY updated_offense_category
ORDER  BY count_offense DESC
LIMIT  5 
"""


t5_d = pd.read_sql_query(t5, cnx)
t5_d

,updated_offense_category,count_offense
0,Narcotics,62837
1,UUW - Unlawful Use of Weapon,29381
2,Retail Theft,13226
3,Burglary,10118
4,Aggravated DUI,9373


In [24]:
## filter to rows where UPDATED_OFFENSE_CATEGORY
## is in the top 5
t5_rowfilt = """
SELECT *
FROM   caseinit
       INNER JOIN (SELECT updated_offense_category AS tmp_oc,
                          Count(*)                 AS count_offense
                   FROM   caseinit
                   WHERE  race IN ( "black", "white" )
                   GROUP  BY updated_offense_category
                   ORDER  BY count_offense DESC
                   LIMIT  5) AS top5
               ON caseinit.updated_offense_category = top5.tmp_oc
WHERE  race IN ( "black", "white" ) 
"""

t5_d = pd.read_sql_query(t5_rowfilt, cnx)
t5_d.shape
t5_d.RACE.value_counts()
t5_d.UPDATED_OFFENSE_CATEGORY.value_counts()

(124935, 45)

RACE
Black    105770
White     19165
Name: count, dtype: int64

UPDATED_OFFENSE_CATEGORY
Narcotics                       62837
UUW - Unlawful Use of Weapon    29381
Retail Theft                    13226
Burglary                        10118
Aggravated DUI                   9373
Name: count, dtype: int64

In [25]:
## get diversion rates by race for white and black
## defendants 
divert_compare = """
SELECT updated_offense_category,
       is_in_diversion,
       race,
       Count(*)               AS count_divert,
       Count(*) / count_group AS prop_divert
FROM   caseinit
       INNER JOIN (SELECT updated_offense_category AS tmp_oc,
                          race                     AS tmp_race,
                          Count(*)                 AS count_group
                   FROM   caseinit
                   WHERE  race IN ( "black", "white" )
                   GROUP  BY updated_offense_category,
                             race) AS tmp
               ON tmp.tmp_race = caseinit.race
                  AND tmp.tmp_oc = caseinit.updated_offense_category
GROUP  BY updated_offense_category,
          race,
          is_in_diversion
ORDER  BY count_divert DESC 
"""

divert_compare_alloff = pd.read_sql_query(divert_compare, cnx)
divert_compare_alloff.head()


,updated_offense_category,is_in_diversion,race,count_divert,prop_divert
0,Narcotics,False,Black,51103,0.9367
1,UUW - Unlawful Use of Weapon,False,Black,28008,0.9979
2,Retail Theft,False,Black,8239,0.8487
3,PROMIS Conversion,False,Black,7277,1.0000
4,Narcotics,False,White,7221,0.8722


In [17]:
## combine with the top 5 filtering code
divert_compare_t5 = """
SELECT updated_offense_category,
       is_in_diversion,
       race,
       Count(*)               AS count_divert,
       Count(*) / count_group AS prop_divert
FROM   caseinit
       INNER JOIN (SELECT updated_offense_category AS tmp_oc,
                          race                     AS tmp_race,
                          Count(*)                 AS count_group
                   FROM   caseinit
                   WHERE  race IN ( "black", "white" )
                   GROUP  BY updated_offense_category,
                             race) AS tmp
               ON tmp.tmp_race = caseinit.race
                  AND tmp.tmp_oc = caseinit.updated_offense_category
       INNER JOIN (SELECT updated_offense_category AS tmp_oc_t5,
                          Count(*)                 AS count_offense
                   FROM   caseinit
                   WHERE  race IN ( "black", "white" )
                   GROUP  BY updated_offense_category
                   ORDER  BY count_offense DESC
                   LIMIT  5) AS top5
               ON caseinit.updated_offense_category = top5.tmp_oc_t5
WHERE  is_in_diversion = 'True'
GROUP  BY updated_offense_category,
          race,
          is_in_diversion 
"""

divert_compare_t5  = pd.read_sql_query(divert_compare_t5, cnx)
divert_compare_t5


,UPDATED_OFFENSE_CATEGORY,is_in_diversion,RACE,count_divert,prop_divert
0,Aggravated DUI,True,White,11,0.0033
1,UUW - Unlawful Use of Weapon,True,Black,58,0.0021
2,Retail Theft,True,White,614,0.1745
3,Retail Theft,True,Black,1469,0.1513
4,Narcotics,True,Black,3455,0.0633
5,Burglary,True,Black,358,0.0482
6,Burglary,True,White,219,0.0813
7,Narcotics,True,White,1058,0.1278
8,UUW - Unlawful Use of Weapon,True,White,9,0.0068
9,Aggravated DUI,True,Black,25,0.0042


# 3. Subqueries and aggregations: two datasets

## 3.1 Looking at structure of diversions dataset

In [19]:
## define a query to pull first 5 rows
## from caseinit table
sample_div_q = """
SELECT *
FROM   divert
LIMIT  5 
"""

divert_res = pd.read_sql_query(sample_div_q, cnx)
divert_res.head()
divert_res.columns

,CASE_ID,CASE_PARTICIPANT_ID,RECEIVED_DATE,OFFENSE_CATEGORY,DIVERSION_PROGRAM,REFERRAL_DATE,DIVERSION_COUNT,PRIMARY_CHARGE_OFFENSE_TITLE,STATUTE,RACE,GENDER,DIVERSION_RESULT,DIVERSION_CLOSED_DATE
0,506061999169,347644693595,01/01/2011 12:00:00 AM,Retail Theft,DS,06/17/2013 12:00:00 AM,2,RETAIL THEFT,720 ILCS 5/16A-3(a),Black,Male,Graduated,06/17/2013 12:00:00 AM
1,506061999169,347644693595,01/01/2011 12:00:00 AM,Retail Theft,VC,08/11/2011 12:00:00 AM,1,RETAIL THEFT,720 ILCS 5/16A-3(a),Black,Male,,
2,506066271711,347649309720,01/02/2011 12:00:00 AM,Retail Theft,MHC,09/14/2012 12:00:00 AM,1,RETAIL THEFT,720 ILCS 5/16A-3(a),Black,Male,Failed,09/14/2012 12:00:00 AM
3,506071709491,347654941393,01/03/2011 12:00:00 AM,Narcotics,MHC,07/27/2018 12:00:00 AM,1,[POSSESSION OF CONTROLLED SUBSTANCE WITH INTEN...,720 ILCS 570/401(d)(i),Black,Female,,
4,506071709491,347654987555,01/03/2011 12:00:00 AM,Narcotics,MHC,07/27/2018 12:00:00 AM,1,POSSESSION OF A CONTROLLED SUBSTANCE,720 ILCS 570/402(c),White,Male,,


Index(['CASE_ID', 'CASE_PARTICIPANT_ID', 'RECEIVED_DATE', 'OFFENSE_CATEGORY',
       'DIVERSION_PROGRAM', 'REFERRAL_DATE', 'DIVERSION_COUNT',
       'PRIMARY_CHARGE_OFFENSE_TITLE', 'STATUTE', 'RACE', 'GENDER',
       'DIVERSION_RESULT', 'DIVERSION_CLOSED_DATE'],
      dtype='object')

## 3.2 Left joining certain cols from caseinit onto diversions

### Left join with everything from divert; certain cols from caseinit

In [20]:
lj_q = """
SELECT divert.*,
       age_at_incident,
       law_enforcement_agency,
       incident_city
FROM   divert
       LEFT JOIN caseinit
              ON divert.case_id = caseinit.case_id
              AND divert.case_participant_id = caseinit.case_participant_id 
"""

In [21]:
lj_d = pd.read_sql_query(lj_q, cnx)
lj_d.head()
lj_d.shape
lj_d.columns

,CASE_ID,CASE_PARTICIPANT_ID,RECEIVED_DATE,OFFENSE_CATEGORY,DIVERSION_PROGRAM,REFERRAL_DATE,DIVERSION_COUNT,PRIMARY_CHARGE_OFFENSE_TITLE,STATUTE,RACE,GENDER,DIVERSION_RESULT,DIVERSION_CLOSED_DATE,AGE_AT_INCIDENT,LAW_ENFORCEMENT_AGENCY,INCIDENT_CITY
0,507323758467,348994448649,02/24/2011 12:00:00 AM,Narcotics,DS,03/16/2011 12:00:00 AM,1,POSSESSION OF A CONTROLLED SUBSTANCE,720 ILCS 570/402(c),Black,Male,Graduated,07/22/2011 12:00:00 AM,None,None,None
1,507920360678,349624088148,03/18/2011 12:00:00 AM,Narcotics,DS,04/13/2011 12:00:00 AM,1,POSSESSION OF A CONTROLLED SUBSTANCE,720 ILCS 570/402(c),Biracial,Female,,,37.0,CHICAGO PD,Chicago
2,508118645462,349817549962,03/25/2011 12:00:00 AM,Credit Card Cases,BR9,03/21/2012 12:00:00 AM,1,MISUSE CREDIT CARD <$300,720 ILCS 250/8,Latinx,Female,Graduated,03/21/2012 12:00:00 AM,None,None,None
3,508232644648,349936830641,03/30/2011 12:00:00 AM,Narcotics,DS,05/02/2011 12:00:00 AM,1,POSSESSION OF CANNABIS,720 ILCS 550/4(d),Black,Male,Graduated,09/02/2011 12:00:00 AM,None,None,None
4,508661258280,350393734729,04/15/2011 12:00:00 AM,Retail Theft,VC,03/23/2012 12:00:00 AM,1,RETAIL THEFT,720 ILCS 5/16A-3(a),Black,Male,Graduated,05/24/2013 12:00:00 AM,None,None,None


(27902, 16)

Index(['CASE_ID', 'CASE_PARTICIPANT_ID', 'RECEIVED_DATE', 'OFFENSE_CATEGORY',
       'DIVERSION_PROGRAM', 'REFERRAL_DATE', 'DIVERSION_COUNT',
       'PRIMARY_CHARGE_OFFENSE_TITLE', 'STATUTE', 'RACE', 'GENDER',
       'DIVERSION_RESULT', 'DIVERSION_CLOSED_DATE', 'AGE_AT_INCIDENT',
       'LAW_ENFORCEMENT_AGENCY', 'INCIDENT_CITY'],
      dtype='object')

In [24]:
lj_dupfix_q = """
SELECT divert.*,
       age_at_incident,
       law_enforcement_agency,
       incident_city,
       caseinit.race AS caseinit_race
FROM   divert
       LEFT JOIN caseinit
              ON divert.case_id = caseinit.case_id
                 AND divert.case_participant_id = caseinit.case_participant_id 
"""

In [25]:
lj_d_dupfix = pd.read_sql_query(lj_dupfix_q, cnx)

In [26]:
## only diff is that in diversion dataset
## they have the category Latinx
## while in case initiations that's Hispanic or 
## White hispanic/latino
pd.crosstab(lj_d_dupfix.RACE, lj_d_dupfix.caseinit_race)

caseinit_race,,American Indian,Asian,Biracial,Black,HISPANIC,Unknown,White,White [Hispanic or Latino],White/Black [Hispanic or Latino]
RACE,,,,,,,,,,
Asian,0,0,78,0,0,0,0,0,0,0
Biracial,0,0,0,1,0,0,0,0,0,57
Black,0,0,0,0,7037,0,0,0,0,0
Latinx,0,0,0,0,0,82,0,0,1169,0
Other,0,4,0,0,0,0,0,0,0,0
Unknown,55,0,0,0,0,0,16,0,0,0
White,0,0,0,0,0,0,0,2736,0,0


### Aliasing the tables in the join

In [27]:
lj_alias_q = """
SELECT d.*,
       age_at_incident,
       law_enforcement_agency,
       incident_city,
       c.race AS caseinit_race
FROM   divert AS d
       LEFT JOIN caseinit AS c
              ON d.case_id = c.case_id
                 AND d.case_participant_id = c.case_participant_id 
"""

In [28]:
lj_alias_d = pd.read_sql_query(lj_alias_q, cnx)

# 4. Combining agg and join

Task: among the cases that are diverted, for each of the charges (`UPDATED_OFFENSE_CATEGORY`) in the case initiations, find the percentage of defendants with that charge going to each `DIVERSION_PROGRAM`

### First part: count of offenses by diversion program

In [29]:
offenses_byprogram = """
SELECT Count(*) AS count_offenses_byprogram,
       updated_offense_category,
       diversion_program
FROM   divert
       INNER JOIN caseinit
               ON divert.case_id = caseinit.case_id
                  AND divert.case_participant_id = caseinit.case_participant_id
GROUP  BY updated_offense_category,
          diversion_program
ORDER  BY count_offenses_byprogram DESC 
"""

In [30]:
cp = pd.read_sql_query(offenses_byprogram, cnx)
cp.head()

cp[cp.UPDATED_OFFENSE_CATEGORY == "Narcotics"]

,count_offenses_byprogram,UPDATED_OFFENSE_CATEGORY,DIVERSION_PROGRAM
0,2149,Narcotics,DC
1,1026,Retail Theft,DC
2,862,Retail Theft,MHC
3,752,Narcotics,DS
4,676,Narcotics,ARI


,count_offenses_byprogram,UPDATED_OFFENSE_CATEGORY,DIVERSION_PROGRAM
0,2149,Narcotics,DC
3,752,Narcotics,DS
4,676,Narcotics,ARI
5,618,Narcotics,DDPP
6,572,Narcotics,MHC
7,466,Narcotics,ACT
11,191,Narcotics,BR9
14,169,Narcotics,VC
59,17,Narcotics,RJCC
62,15,Narcotics,SEED


### Second part: count of offenses total

In [31]:
offenses_total = """
SELECT Count(*) AS count_offenses_total,
       updated_offense_category
FROM   divert AS d
       INNER JOIN caseinit AS c
               ON d.case_id = c.case_id
                  AND d.case_participant_id = c.case_participant_id
GROUP  BY updated_offense_category
ORDER  BY count_offenses_total DESC 
"""

In [32]:
c = pd.read_sql_query(offenses_total, cnx)
c.head()

## check that total matches sum from first query- see both are 520 
cp.count_offenses_byprogram[cp.UPDATED_OFFENSE_CATEGORY == "Narcotics"].sum()

,count_offenses_total,UPDATED_OFFENSE_CATEGORY
0,5625,Narcotics
1,2497,Retail Theft
2,742,Burglary
3,427,Theft
4,218,Forgery


5625

### Combining into one query 

Put the numerator into a subquery and do the proportions in the outer query. We add the second inner join to make sure that when we get the denominator from the outer part, we're restricting to defendants sent to diversion

In [33]:
offenses_prop = """
SELECT count_offenses_byprogram / Count(*) AS prop_offenses_byprogram,
       caseinit.updated_offense_category,
       diversion_program
FROM   caseinit
       INNER JOIN (SELECT Count(*) AS count_offenses_byprogram,
                          updated_offense_category,
                          diversion_program
                   FROM   divert
                          INNER JOIN caseinit
                                  ON divert.case_id = caseinit.case_id
                                     AND divert.case_participant_id =
                                         caseinit.case_participant_id
                   GROUP  BY updated_offense_category,
                             diversion_program) AS num
               ON num.updated_offense_category =
                  caseinit.updated_offense_category
       INNER JOIN (SELECT case_id             AS cid,
                          case_participant_id AS cpid
                   FROM   divert) AS ppl_divert
               ON caseinit.case_id = ppl_divert.cid
                  AND caseinit.case_participant_id = ppl_divert.cpid
GROUP  BY caseinit.updated_offense_category,
          diversion_program 
"""

## note: check the denom

In [34]:
cd_prop = pd.read_sql_query(offenses_prop, cnx)



In [35]:
cd_prop.head()
cd_prop[cd_prop.UPDATED_OFFENSE_CATEGORY == "Narcotics"]

## confirm with above counts- see ddpp matches up! :)
prop_ddpp = cp.count_offenses_byprogram[(cp.UPDATED_OFFENSE_CATEGORY == "Narcotics") &
                                       (cp.DIVERSION_PROGRAM == "DDPP")].iloc[0]/ \
            c.count_offenses_total[c.UPDATED_OFFENSE_CATEGORY == "Narcotics"].iloc[0]
prop_ddpp

,prop_offenses_byprogram,UPDATED_OFFENSE_CATEGORY,DIVERSION_PROGRAM
0,0.0485,Retail Theft,VC
1,0.0044,Retail Theft,DS
2,0.3452,Retail Theft,MHC
3,0.1017,Narcotics,MHC
4,0.1337,Narcotics,DS


,prop_offenses_byprogram,UPDATED_OFFENSE_CATEGORY,DIVERSION_PROGRAM
3,0.1017,Narcotics,MHC
4,0.1337,Narcotics,DS
5,0.3820,Narcotics,DC
7,0.1202,Narcotics,ARI
10,0.0340,Narcotics,BR9
14,0.0300,Narcotics,VC
94,0.0828,Narcotics,ACT
98,0.1099,Narcotics,DDPP
105,0.0030,Narcotics,RJCC
117,0.0027,Narcotics,SEED


0.10986666666666667